In [1]:
import paddle.nn as nn
import paddle
from paddle.fluid.layer_helper import LayerHelper
from paddle.io import Dataset,DataLoader
import glob
import os
from tqdm import tqdm
import matplotlib.image as mpimg
import cv2
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import backbone
import Danet
import dataset

/Library/Python/3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
net=Danet.DANet('danet')

In [ ]:
epoch=15
batch_num = 32
Learning_rate = 1e-2
end_lr=1e-4
lr_power=0.9
l2decay=0.0001
momentum=0.9
acc_list=[0]

lr=paddle.optimizer.lr.PolynomialDecay(Learning_rate,epoch*batch_num,end_lr,lr_power)
l2_decay=paddle.regularizer.L2Decay(l2decay)

dataset=dataset.CitySegmentation()
dataloader = DataLoader(dataset,batch_size=batch_num,num_workers=2,shuffle=True)

warnings.filterwarnings("ignore")

#criterion = nn.functional.softmax_with_cross_entropy
criterion = nn.BCELoss()

#optim = paddle.optimizer.Adam(parameters=net.parameters(),learning_rate=Learning_rate)
optim = paddle.optimizer.Momentum(lr,momentum=momentum,parameters=net.parameters(),weight_decay=l2_decay)

item=paddle.load("net.pdparams")
net.set_state_dict(item)

for epoch in range(epoch):
    print("epoch",epoch)
    for i,(x,y) in enumerate(tqdm(dataloader)):
        x=paddle.to_tensor(x)

        y[y.numpy()<0.5]=0
        y[y.numpy()>=0.5]=1
        y=paddle.to_tensor(y)

        p_out,c_out,output=net(x)

        output=output.squeeze()

        p_out=nn.functional.sigmoid(p_out)
        c_out=nn.functional.sigmoid(c_out)
        output=nn.functional.sigmoid(output)

        loss_p = criterion(p_out, y)
        loss_c = criterion(c_out, y)
        loss_sum = criterion(output, y)
        loss = 0.3*loss_p+0.3*loss_c+0.4*loss_sum

        loss_mean = paddle.mean(loss)

        loss_mean.backward()

        optim.step()
        optim.clear_grad()

        if i%10==0:
            pres = np.argmax(output.numpy(), axis=1)
            gts = np.argmax(y, axis=1)
            correct = (pres == gts).sum().astype('float32')
            acc = correct / (gts.shape[0] * gts.shape[1] * gts.shape[2])
            print(acc)
            print(loss_mean.item())
            if acc>max(acc_list):
                acc_list.append(acc)
                print("max")
                paddle.save(net.state_dict(), "net.pdparams")
paddle.save(net.state_dict(), "net.pdparams")
